<b>Исследование данных о бронировании отелей</b>
<u>Шаг 1. Откройте файл с данными и изучите общую информацию</u>

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
bookings = pd.read_csv(r'C:\Users\user\Desktop\practice\2_bookings.csv',
                       encoding='windows-1251', sep=';')
bookings.info()

<class 'pandas.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  str    
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  str    
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  str    
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     119390 non-nu

<b>Выводы</b>  
Рассмотрим полученную информацию подробнее.  
Всего в таблице 21 столбец 119390 строк, типы данных - float64, int64, str 
Описание данных:  
• Hotel – тип отеля (City Hotel или Resort Hotel) 
• Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
• Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
• Arrival full date – полная дата прибытия
• Arrival date year – год прибытия  
• Arrival date month – месяц прибытия  
• Arrival date week number – номер недели прибытия
• Arrival date day of month – день прибытия
• Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость
забронировал для проживания в отеле
• Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал
для проживания в отеле
• Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
• Adults – число взрослых
• Children – число детей
• Babies – число младенцев 
• Meal – выбранный тип питания
• Country – страна происхождения клиента
• Reserved room type – тип зарезервированного номера
• Assigned room type – тип полученного номера (может отличаться от забронированного)
• Customer type – тип бронирования
• Reservation status – значение последнего статуса брони: Canceled - было отменено
клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не
зарегистрировался и сообщил администрации отеля причину
• Reservation status date – дата обновления статуса  

Количество значений в столбцах отличается. Это говорит о том, что в данных есть пропущенные значения (колонки Children, Country).  
Для начала приведем названия столбцов к более удобному для дальнейшей работы формату, но пропуски оставим.  

<u>Шаг 2. Предобработка данных и вывод топ-5 стран с наибольшим числом успешных бронирований</u>

In [118]:
bookings.columns = (bookings.columns.str.lower()
                                    .str.replace(' ', '_'))
bookings.loc[bookings.is_canceled == 0] \
        .country.value_counts() \
        .head()  # отбирает строки, удовлетворяющие условию "бронь не отменена", и считает количество таких броней для всех стран, выводя топ-5

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: count, dtype: int64

<b>Выводы</b>  
Названия столбцов приведены к единому формату.  
Португалия (PRT) лидирует по количеству успешных бронирований — 21 071 запись. Это в ~2,2 раза больше, чем у страны на втором месте (Великобритания, 9 676 записей).  

<u>Шаг 3. Определение среднего количества ночей, на которое бронируют отели разных типов</u>

In [119]:
bookings.groupby(bookings.hotel, as_index=False) \
        .stays_total_nights \
        .mean() \
        .round(2) # группировка по типу отеля, поиск среднего количества забронированных ночей и округление полученных значений

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


<b>Выводы</b>  
Гости Resort Hotel в среднем снимают номера на ~45% ночей дольше, чем гости City Hotel, что может говорить о разных целевых аудиториях данных типов отелей (Resort Hotel - курортный отель).  

<u>Шаг 4. Определение количества кейсов, когда тип номера, присвоенный клиенту, отличается от изначально забронированного</u>

In [134]:
other_room = len(bookings.loc[(bookings.assigned_room_type != bookings.reserved_room_type) &
                              (bookings.is_canceled == 0)])
print('Количество наблюдений:', other_room)

Количество наблюдений: 14115


<b>Выводы</b>  
Подобные наблюдения, когда номер отличался от первоначального, имеются, но их относительно не много - около 12% (14 115 записей).  

<u>Шаг 5. Месяц, на который чаще всего оформляли бронь в 2016 и 2017 гг</u>

In [97]:
bookings.loc[bookings.arrival_date_year.isin([2016, 2017]), ['arrival_date_year', 'arrival_date_month']] \
         .value_counts() \
         .head(2)

arrival_date_year  arrival_date_month
2017               May                   6313
2016               October               6203
Name: count, dtype: int64

<b>Выводы</b>  
В 2016 году гости чаще предпочитали заселяться осенью, а в следующем году большинство посетителей выбирали весну.  

<u>Шаг 6. Месяц, на который чаще всего отменялись бронирования отеля типа City Hotel в 2015-2017 гг</u>

In [98]:
result = (bookings.loc[bookings.hotel == 'City Hotel'] 
         .groupby([bookings.arrival_date_year, bookings.arrival_date_month], as_index = False)
         .agg({'is_canceled' : 'sum'}))
result = result.loc[result.groupby('arrival_date_year')['is_canceled']
                          .idxmax()] # индекс максимального числа по столбцу счетчика отмен
print(result)

    arrival_date_year arrival_date_month  is_canceled
5                2015          September         1543
16               2016            October         1947
25               2017                May         2217


<u>Шаг 7. Каких гостей в среднем больше: взрослых, детей или младенцев? </u>

In [137]:
max_avg = bookings[['adults', 'children', 'babies']].mean() 
print(f'{max_avg.idxmax()}: {max_avg.max().round(2)}')

adults: 1.86


<u>Шаг 8. Кто чаще заселяется в отели разных типов: дети или младенцы? </u>

In [141]:
bookings['total_kids'] = bookings[['children', 'babies']].sum(axis=1) # запись суммы кол-ва детей и младенцев в доп.столбец
bookings.groupby(['hotel']) \
        .total_kids.mean() \
        .round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

<u>Шаг 9. Расчет коэффициента оттока в зависимости от наличия детей у гостей (CR) </u>

In [152]:
bookings['has_kids'] = bookings.total_kids.astype(bool)
bookings[['has_kids', 'is_canceled']].value_counts() # кол-во строк в разбивке на "с детьми/без детей" и "была/не была отмена брони"
bookings[['has_kids']].value_counts() # разбивка на "с детьми/без детей"
(bookings[['has_kids', 'is_canceled']].value_counts() / bookings[['has_kids']].value_counts() * 100).round(2)

has_kids  is_canceled
False     0              62.78
          1              37.22
True      0              65.08
          1              34.92
Name: count, dtype: float64

<b>Выводы</b>  
Гости с детьми отменяли бронирование, что может говорить о том, что семьи - более лояльная аудитория. Одним из вариантов стимулирования данного сегмента бронировать номера чаще может быть запуск таргетированных акций (скидки на длительное проживание, бесплатное питание для детей и т.д.)